Tensorflow后端会自动构建计算图。通过计算图，系统可以知道所有计算的依赖关系，并可以**选择将没有依赖关系的多个任务并行执行来获得计算性能的提升**。例如“异步计算”一节的第一个例子里依次执行了a = tf.ones((1, 2))和b = tf.ones((1, 2))。这两步计算之间并没有依赖关系，因此系统可以选择并行执行它们。

通常，一个运算符会用到所有CPU或单块GPU上全部的计算资源。例如，dot运算符会用到所有CPU（即使是一台机器上有多个CPU处理器）或单块GPU上所有的线程。如果每个运算符的计算量足够大，只在CPU上或者单块GPU上并行运行多个运算符时，每个运算符的运行只分到CPU或单块GPU上部分计算资源。即使这些计算可以并行，最终计算性能的提升可能也并不明显。本节中探讨的自动并行计算**主要关注同时使用CPU和GPU的并行计算，以及计算和通信的并行**。

首先导入本节中实验所需的包或模块。注意，需要至少一块GPU才能运行本节实验。

In [2]:
import tensorflow as tf
import time

## 8.3.1 CPU和GPU的并行计算

我们先介绍CPU和GPU的并行计算，例如，程序中的计算既发生在CPU上，又发生在GPU上。先定义`run`函数，令它做10次矩阵乘法。

In [1]:
def run(x):
    return [tf.matmul(x, x) for _ in range(10)]

接下来，分别在CPU和GPU上创建`Tensor`。

In [3]:
with tf.device('/CPU:0'):
    x_cpu = tf.random.uniform(shape=(2000, 2000))

In [4]:
with tf.device('/GPU:0'):
    x_gpu = tf.random.uniform(shape=(6000, 6000))

然后，分别使用它们在CPU和GPU上运行`run`函数并打印运行所需时间。

In [5]:
run(x_cpu)

[<tf.Tensor: shape=(2000, 2000), dtype=float32, numpy=
 array([[500.9258 , 504.29004, 498.69098, ..., 487.86374, 499.3651 ,
         506.09497],
        [509.98218, 507.78857, 506.74323, ..., 493.0764 , 501.04803,
         516.38947],
        [498.52655, 496.70752, 493.25797, ..., 480.53812, 489.60587,
         494.6193 ],
        ...,
        [508.1    , 506.99084, 505.30087, ..., 490.87817, 502.99252,
         507.55746],
        [501.1965 , 498.27576, 497.39856, ..., 488.7976 , 498.4595 ,
         505.22424],
        [492.8983 , 496.93564, 494.66882, ..., 483.37897, 484.44055,
         497.0672 ]], dtype=float32)>,
 <tf.Tensor: shape=(2000, 2000), dtype=float32, numpy=
 array([[500.9258 , 504.29004, 498.69098, ..., 487.86374, 499.3651 ,
         506.09497],
        [509.98218, 507.78857, 506.74323, ..., 493.0764 , 501.04803,
         516.38947],
        [498.52655, 496.70752, 493.25797, ..., 480.53812, 489.60587,
         494.6193 ],
        ...,
        [508.1    , 506.99084, 505.3

In [6]:
run(x_gpu)

[<tf.Tensor: shape=(6000, 6000), dtype=float32, numpy=
 array([[1502.1893, 1504.2753, 1482.5637, ..., 1494.0295, 1478.1432,
         1501.2893],
        [1500.0691, 1500.858 , 1503.4851, ..., 1499.0657, 1484.0063,
         1502.2512],
        [1503.4503, 1505.9158, 1481.8478, ..., 1495.8295, 1486.4708,
         1513.1494],
        ...,
        [1498.8539, 1513.1327, 1500.6642, ..., 1505.8422, 1510.3264,
         1510.6282],
        [1496.5533, 1497.2009, 1484.2028, ..., 1487.465 , 1490.977 ,
         1501.5931],
        [1538.3323, 1532.3579, 1511.4791, ..., 1532.1012, 1527.6675,
         1532.39  ]], dtype=float32)>,
 <tf.Tensor: shape=(6000, 6000), dtype=float32, numpy=
 array([[1502.1893, 1504.2753, 1482.5637, ..., 1494.0295, 1478.1432,
         1501.2893],
        [1500.0691, 1500.858 , 1503.4851, ..., 1499.0657, 1484.0063,
         1502.2512],
        [1503.4503, 1505.9158, 1481.8478, ..., 1495.8295, 1486.4708,
         1513.1494],
        ...,
        [1498.8539, 1513.1327, 1500.

In [8]:
class Benchmark(object):
    def __init__(self, prefix=None):
        self.prefix = prefix + ' ' if prefix else ''

    def __enter__(self):
        self.start = time.time()

    def __exit__(self, *args):
        print('%stime: %.4f sec' % (self.prefix, time.time() - self.start))

In [9]:
with Benchmark('Run on CPU.'):
    run(x_cpu)

Run on CPU. time: 0.5163 sec


In [10]:
with Benchmark('Then Run on GPU.'):
    run(x_gpu)

Then Run on GPU. time: 16.3127 sec


**核心显卡的计算能力着实有限。**

尝试系统能自动并行这两个任务：

In [11]:
with Benchmark('Run on both CPU and GPU in parallel.'):
    run(x_cpu)
    run(x_gpu)

Run on both CPU and GPU in parallel. time: 17.0455 sec


可以看到，当两个计算任务一起执行时，执行总时间小于它们分开执行的总和。这表明，Tensorflow能有效地在CPU和GPU上自动并行计算。

**当GPU比较好时可能会，现在这台电脑的GPU就。。**

## 8.3.2 计算和通信的并行计算

在同时使用CPU和GPU的计算中，**经常需要在内存和显存之间复制数据，造成数据的通信**。在下面的例子中，**我们在GPU上计算，然后将结果复制回CPU使用的内存**。我们分别打印GPU上计算时间和显存到内存的通信时间。

In [13]:
def copy_to_cpu(x):
    with tf.device('/CPU:0'):
        return [y for y in x]

with Benchmark('Run on GPU.'):
    y = run(x_gpu)

with Benchmark('Then copy to CPU.'):
    copy_to_cpu(y)

Run on GPU. time: 16.0555 sec
Then copy to CPU. time: 0.0010 sec


In [14]:
with Benchmark('Run on GPU.'):
    y = run(x_gpu)

Run on GPU. time: 16.4707 sec


In [15]:
with Benchmark('Then copy to CPU.'):
    copy_to_cpu(y)

Then copy to CPU. time: 0.0000 sec


打印这两个任务完成的总时间。

In [16]:
with Benchmark('Run and copy in parallel.'):
    y = run(x_gpu)
    copy_to_cpu(y)

Run and copy in parallel. time: 17.0351 sec


**结论：GPU的运行速度会很大程度影响上述方法的使用。**

> 注：本节与原书有很多不同，[原书传送门](https://zh.d2l.ai/chapter_computational-performance/auto-parallelism.html)

